In [2]:
import fitz  # PyMuPDF
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# PDF 파일 열기 및 텍스트 추출
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    pdf_text = []
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        pdf_text.append(page.get_text())
    return pdf_text

# 텍스트 청크 나누기
def chunk_text(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# BLEU Score 계산
def calculate_bleu(reference, hypothesis):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference], hypothesis, smoothing_function=smoothie)

# JSON 파일 읽기
def load_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# PDF에서 텍스트 추출
pdf_text = extract_text_from_pdf("D:/[24]ICT_Practice/practice_file/(2024)포트미스_가이드북.pdf")

# 텍스트를 청크 단위로 나누기
pdf_chunks = [chunk_text(page) for page in pdf_text]

# JSON 데이터 로드
qa_data = load_json("D:/[24]ICT_Practice/fineturing_ex/dataset/(2024)gpt4o_fineturning_data.json")

# BLEU Score 계산
results = []

for entry in qa_data:
    question = entry["QUESTION"]
    answer = entry["ANSWER"]
    
    # 모든 청크에서 정답과 일치하는 부분 찾기
    found = False
    for page_chunks in pdf_chunks:
        for chunk in page_chunks:
            if answer in chunk:
                reference = chunk
                hypothesis = answer
                bleu_score = calculate_bleu(reference, hypothesis)
                results.append({
                    "question": question,
                    "answer": answer,
                    "bleu_score": bleu_score
                })
                found = True
                break
        if found:
            break
    if not found:
        results.append({
            "question": question,
            "answer": answer,
            "bleu_score": 0.0
        })

# 결과 출력
for result in results:
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer']}")
    print(f"BLEU Score: {result['bleu_score']}\n")


Question: 가이드북글로벌 리더로 도약하기 위한 문구는 무엇입니까?
Answer: 가이드북글로벌 리더로 도약하기 위한
BLEU Score: 0.0

Question: 해운, 조선 상생을 통한 해운강국 건설을 위해 어떤 목표를 실현하고자 합니까?
Answer: 해운, 조선 상생을 통한 해운강국 건설을 위해 세계 해운산업 리더국가 실현과 우리나라 항만을 글로벌 물류 중심지로 만드는 목표를 실현하고자 합니다.
BLEU Score: 0.0

Question: 포트미스의 주요 서비스 중 항만물류와 관련된 통계 항목은 무엇이 있습니까?
Answer: 포트미스의 주요 서비스 중 항만물류와 관련된 통계 항목에는 컨테이너 통계, 화물 통계, 선박 입출항 통계, 수출입 물류 통계가 있습니다.
BLEU Score: 0.0

Question: 포트미스 이용 환경에서 모바일서비스를 통해 제공되는 기능은 무엇인가요?
Answer: 모바일서비스를 통해 민원신고 및 신고 결과 조회, 선박에 대한 관제 현황 정보 조회, EDI 및 ebXML 응답문서 조회 기능이 제공됩니다.
BLEU Score: 0.0

Question: 포트미스 회원가입 절차에서 사용 업체 등록 신청 시 필요한 서류는 무엇입니까?
Answer: 사용 업체 등록 신청 시 필요한 서류는 사업자 등록증입니다.
BLEU Score: 0.0

Question: 포트미스에서 선박제원 신청 절차 중 오류가 발생했을 때 민원인은 어떻게 해야 하나요?
Answer: 민원인은 오류 내용을 확인 후 신청정보를 수정하여 재신고해야 합니다.
BLEU Score: 0.0

Question: 외항선의 입항 절차에서 '선박평형수신고'는 어느 단계에서 이루어지나요?
Answer: '선박평형수신고'는 외항입항신고 절차에서 담당자가 수리하는 단계에서 이루어집니다.
BLEU Score: 0.0

Question: 저속운항선박 지원신청과 인센티브 신청은 각각 어떤 번호로 구분되나요?
Answer: 저속운항선박 지원신청은 5-1로, 저속운항선박 인센티브 

In [4]:
# 청크 확인 단계 추가
for page_num, chunks in enumerate(pdf_chunks):
    print(f"\nPage {page_num + 1}:")
    for chunk_num, chunk in enumerate(chunks):
        print(f"\nChunk {chunk_num + 1}:\n{chunk[:500]}...")  # 첫 500글자만 출력


Page 1:

Chunk 1:
가이드북
글로벌 리더로 도약하기 위한
...

Page 2:

Chunk 1:
글로벌 물류 중심
대한민국
해운산업
리더국가
고객과 
함께하는 
미래
해운업 
상생을 통한
해운강국 건설
가이드북
글로벌 리더로 도약하기 위한
CONTENTS
세계 해운산업 리더국가 실현
우리나라 항만을 글로벌 물류 중심지로
해운, 조선 상생을 통한 해운강국 건설
04
06
08
10
12
14
16
18
20
22
24
26
28
포트미스 주요 서비스
포트미스 이용 환경
포트미스 회원가입 절차
입·출항 선박에 대한 선박제원 등록
(변경) 절차 업무 흐름
외항선 민원신고 절차
내항선 민원신고 절차
외항선입항(출항)신고(최초,변경)
승무원/승객명부
강제도선 면제신청
예선사용 면제신청
선박평형수신고
선박보안정보통보신고
저속운항선박지원신청
저속운항선박 인센티브 신청
항만시설사용허가신청서(선석신청)
항만시설사용료(정박료)면제신청
30
32
34
36
38
40
42
44
48
58
80
위험물 반입신고 /
위험물 일람표신고
통합화물신고
외항입항(출항)신고(최종)
항만시설사용신고(화물료)
화...

Page 3:

Chunk 1:
해운산업 리더국가 실현
글로벌 경갱력을 갖춘 고부가가치 스마트 항만 실현
우리나라 항만을 글로벌 물류 중심지로!
포트미스 주요 서비스
해운민원
선박용물건
승하선공인
운임공표
등록
선원업무
해운사업자
선박업무
항만민원
관제
예도선
사용료징수
화물/위험물
항만시설
선박운항
항만물류
통계
컨테이너
통계
화물통계
선박입출항
통계
수출입물류
통계
04 • PORT-MIS
05
...

Page 4:

Chunk 1:
포트미스 이용 환경
내항신고 위주 신고 적합 
이용요금: 무료
항만에 관련된 모든 신고문서를 처리 
단건 신고시 적합
이용요금: 무료
신청건이 많은 신고 문서에 적합
(예: 화물신고 및 컨테이너신고, 위험물 일람표 신고 등)
이용요금: 사용 데이터량에 따라 이용요금 발생(유료)
모바일서비스
인터넷 웹 서비스
KL-NET
PLIS